In [12]:
import numpy as np
import random,math
import os

import matplotlib.pyplot as plt

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

directory = 'LBPTOP_npy/Drunk'
Drunk=[]
for filename in os.listdir(directory):
    Drunk.append(os.path.join(directory, filename))

directory = 'LBPTOP_npy/Sober'
Sober=[]
for filename in os.listdir(directory):
    Sober.append(os.path.join(directory, filename))

Drunk_data = []
for filename in Drunk:
    file = np.load(filename)
    Drunk_data.append(file)

Drunk_data = np.array(Drunk_data)

Sober_data = []
for filename in Sober:
    file = np.load(filename) 
    Sober_data.append(file)

Sober_data = np.array(Sober_data)

X = np.concatenate((Drunk_data , Sober_data))
zeros = np.zeros(len(Drunk_data))
ones = np.ones(len(Sober_data))
y = np.concatenate((zeros, ones))

c = list(zip(X, y))
random.shuffle(c)
X, y = zip(*c)

#print(len(X),len(y))
fraction = 0.85 #size of train

num_of_Train_exp = math.floor(len(X)*(fraction))
print(len(X),len(y),num_of_Train_exp)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

X_train = X[0:num_of_Train_exp]
X_test = X[num_of_Train_exp:-1]
y_train = y[0:num_of_Train_exp]
y_test = y[num_of_Train_exp:-1]
print(len(X_train),len(X_test),len(y_train),len(y_test))

3118 3118 2650
2650 467 2650 467


In [13]:
def evaluate_on_test_data(model=None):
    predictions = model.predict(X_test)
    correct_classifications = 0
    for i in range(len(y_test)):
        if predictions[i] == y_test[i]:
            correct_classifications += 1
    accuracy = 100*correct_classifications/len(y_test) #Accuracy as a percentage
    return accuracy


In [14]:
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),algorithm="SAMME",n_estimators=200)
bdt.fit(X_train, y_train)
acc = evaluate_on_test_data(bdt)
print("{} % accuracy obtained AdaBoost".format(acc))
expected = y_test
predicted = bdt.predict(X_test)
print("Classification report for classifier %s:\n%s\n"% (bdt, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))


86.7237687366167 % accuracy obtained with kernel = rbf
Classification report for classifier AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=200, random_state=None):
             precision    recall  f1-score   support

        0.0       0.87      0.81      0.84       196
        1.0       0.87      0.91      0.89       271

avg / total       0.87      0.87      0.87       467


Confusion matrix:
[[158  38]
 [ 24 247]]
